In [8]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime
import threading

import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price, convert_lots_to_usd
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import CURRENCY_PAIRS, MT5Timeframe

In [9]:
# PAIRS = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
PAIRS = ['BITCOIN', 'ETHEREUM', 'SOLANA', 'DOGECOIN', 'LITECOIN', 'RIPPLE', 'BNB', 'UNISWAP', 'AVALANCH']
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
LEVERAGE = 500
DEVIATION = 20
VOLUME = 0.1

TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.06, 'new_sl_pnl_multiplier': 0.025},
]

In [10]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

In [11]:
def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

In [12]:
# dict to store trades, keys are position tickets, values are the entire position object
trades = {}

In [13]:
# Utility Functions
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

# Trading Logic Functions
def check_fractal_signals():
    while True:
        for pair in PAIRS:
            if have_open_positions_in_symbol(pair):
                continue

            df = fetch_data_pos(pair, MAIN_TIMEFRAME, 50)
            df['fractal'] = high_low_finder(df)
            df['symbol'] = pair
            display(df.tail())

            price = mt5.symbol_info_tick(pair).ask
            new_trade_capital = calculate_trade_capital(pair, VOLUME, LEVERAGE, price)
            trade_size = calculate_position_size(new_trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 

            last_row = df.iloc[-1]
            current_time = datetime.now().replace(microsecond=0)


            if df['fractal'].iloc[-1] == 'top':
                print(f"{current_time} - {pair} - ENTRY CONDITION - TOP FRACTAL DETECTED - CLOSE: ${last_row['close']:.3f}")
                sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='short')
                order = send_market_order(symbol=pair, volume=VOLUME, order_type='sell', sl=sl, deviation=DEVIATION)
                print(f'{current_time} - {pair} - OPENED POSITION - SHORT - PRICE: {price:.3f} - SL: {sl:.3f}')
            elif df['fractal'].iloc[-1] == 'bottom':
                print(f"{current_time} - {pair} - ENTRY CONDITION - BOTTOM FRACTAL DETECTED - CLOSE: ${last_row['close']:.3f}")
                sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='long')
                order = send_market_order(symbol=pair, volume=VOLUME, order_type='buy', sl=sl, deviation=DEVIATION)
                print(f'{current_time} - {pair} - OPENED POSITION - LONG - PRICE: {price:.3f} - SL: {sl:.3f}')
            # else:
                # print(f'{current_time} - {pair} - NO FRACTAL DETECTED')

        # Wait until the next 15-minute interval
        now = current_time
        next_run = (now.replace(second=0, microsecond=0) + pd.Timedelta(minutes=15 - now.minute % 15)).replace(second=0, microsecond=0)
        next_run += pd.Timedelta(seconds=2)  # Add 2 seconds

        sleep_time = (next_run - now).total_seconds()
        print(f"Waiting for {sleep_time} seconds until next 15-minute check.")
        sleep(sleep_time)

def monitor_open_trades():
    while True:
        current_time = datetime.now().replace(microsecond=0)

        positions = get_positions()
        # print(f'{current_time} - There are {len(positions)} positions open')
        positions['time'] = pd.to_datetime(positions['time'], unit='s')
        positions['time_update'] = pd.to_datetime(positions['time_update'], unit='s')

        for index, position in positions.iterrows():
            # Check if the position ticket exists in trades dict
            if position.ticket not in trades:
                trades[position.ticket] = position
            else:
                # Check if position has changed
                if not trades[position.ticket].equals(position):
                    trades[position.ticket] = position
            
            
            # Calculate the actual capital used for this trade
            trade_capital = calculate_trade_capital(position.symbol, position.volume, LEVERAGE, position.price_open)
            trade_size = calculate_position_size(trade_capital, LEVERAGE)
            fee = calculate_fee(position_size_usd=trade_size) 
            current_pnl_percentage = (position.profit / trade_capital) * 100
            current_sl_pnl = get_pnl_at_price(position.sl, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')
            current_sl_pnl_percentage = (current_sl_pnl / trade_capital) * 100
            # print(f'{position.time_update} - {position.symbol} - POSITION INFO - CURRENT PRICE: ${position.price_current:.3f} - OPEN PRICE: ${position.price_open:.3f} - TRADE CAPITAL: ${trade_capital:.3f} - CURRENT SL: ${position.sl:.3f}')
            # print(f'{position.time_update} - {position.symbol} - PNL INFO - PNL: ${position.profit:.3f} or {current_pnl_percentage:.3f}% - PNL AT CURRENT SL: ${current_sl_pnl:.3f} or {current_sl_pnl_percentage:.3f}%')

            for trailing_step in TRAILING_STOP_STEPS:
                trigger_pnl_multiplier = trailing_step['trigger_pnl_multiplier']
                new_sl_pnl_multiplier = trailing_step['new_sl_pnl_multiplier']

                price_at_trigger_pnl_multiplier = get_price_at_pnl(
                    pnl_multiplier=trigger_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )
                new_sl_at_new_sl_pnl_multiplier = get_price_at_pnl(
                    pnl_multiplier=new_sl_pnl_multiplier,
                    order_fee=fee,
                    position_size_usd=trade_size,
                    leverage=LEVERAGE,
                    entry_price=position.price_open,
                    type='long' if position.type == 0 else 'short'
                )
                pnl_at_trigger = get_pnl_at_price(
                    price_at_trigger_pnl_multiplier,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )
                pnl_at_new_sl = get_pnl_at_price(
                    new_sl_at_new_sl_pnl_multiplier,
                    position.price_open,
                    trade_size,
                    LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )

                pnl_threshold = trigger_pnl_multiplier * trade_capital
                # print(f'{position.time_update} - {position.symbol} - CHECKING TRAILING SL STEP - TRIGGER: {trigger_pnl_multiplier:.3f} (${price_at_trigger_pnl_multiplier:.3f}) (${pnl_at_trigger:.3f} PNL) - NEW SL: {new_sl_pnl_multiplier:.3f} (${new_sl_at_new_sl_pnl_multiplier:.3f}) (${pnl_at_new_sl:.3f} PNL)')

                if position.profit >= pnl_threshold:
                    old_sl_price = position.sl

                    if position.type == 0:  # Long
                        new_sl_price = get_price_at_pnl(
                            pnl_multiplier=new_sl_pnl_multiplier,
                            order_fee=fee,
                            position_size_usd=trade_capital,
                            leverage=LEVERAGE,
                            entry_price=position.price_open,
                            type='long'
                        )
                        if new_sl_price > old_sl_price:
                            print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                            modify_sl_tp(position.ticket, new_sl_price, position.tp)
                    else:  # Short
                        new_sl_price = get_price_at_pnl(
                            pnl_multiplier=new_sl_pnl_multiplier,
                            order_fee=fee,
                            position_size_usd=trade_capital,
                            leverage=LEVERAGE,
                            entry_price=position.price_open,
                            type='short'
                        )
                        if new_sl_price < old_sl_price:
                            print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                            modify_sl_tp(position.ticket, new_sl_price, position.tp)
                    
                    break        

        # Loop through our trades to see if they are not present in position meaning they have been closed
        for trade_ticket, trade in trades.items():
            if trade_ticket not in positions['ticket'].values:
                print(f'{datetime.now()} - {trade.symbol} - POSITION CLOSED - TICKET: {trade_ticket} - PNL: ${trade.profit:.3f}')
                del trades[trade_ticket]

        sleep(10)

In [14]:
# Starting Threads
if __name__ == "__main__":
    fractal_thread = threading.Thread(target=check_fractal_signals, daemon=True)
    monitor_thread = threading.Thread(target=monitor_open_trades, daemon=True)

    fractal_thread.start()
    monitor_thread.start()

    try:
        while True:
            sleep(1)
    except KeyboardInterrupt:
        print("Program terminated by user.")

,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 10:45:00,61396.68,61514.80,61385.85,61438.69,555,0,0,top,BITCOIN
46,2024-10-04 11:00:00,61436.87,61462.06,61336.51,61404.19,618,0,0,bottom,BITCOIN
47,2024-10-04 11:15:00,61405.68,61635.99,61342.19,61563.87,726,0,0,0,BITCOIN
48,2024-10-04 11:30:00,61564.87,61646.57,61522.04,61581.20,751,0,0,top,BITCOIN
49,2024-10-04 11:45:00,61586.66,61601.43,61461.90,61471.58,575,0,0,0,BITCOIN


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 10:45:00,2383.30,2385.69,2379.99,2381.29,644,0,0,top,ETHEREUM
46,2024-10-04 11:00:00,2381.33,2384.50,2377.81,2384.13,688,0,0,bottom,ETHEREUM
47,2024-10-04 11:15:00,2384.30,2393.63,2381.36,2388.63,785,0,0,top,ETHEREUM
48,2024-10-04 11:30:00,2389.19,2392.56,2387.66,2387.81,714,0,0,0,ETHEREUM
49,2024-10-04 11:45:00,2387.81,2387.94,2380.56,2380.92,737,0,0,0,ETHEREUM


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 10:45:00,139.25,139.66,139.06,139.48,231,9,0,0,SOLANA
46,2024-10-04 11:00:00,139.46,140.17,139.12,140.14,304,9,0,0,SOLANA
47,2024-10-04 11:15:00,140.13,140.83,139.89,140.30,520,22,0,0,SOLANA
48,2024-10-04 11:30:00,140.31,140.88,140.16,140.56,482,14,0,top,SOLANA
49,2024-10-04 11:45:00,140.50,140.57,139.90,139.97,352,13,0,0,SOLANA


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 10:45:00,0.10758,0.10789,0.10748,0.10773,282,61,0,top,DOGECOIN
46,2024-10-04 11:00:00,0.10772,0.10782,0.10746,0.10772,318,60,0,bottom,DOGECOIN
47,2024-10-04 11:15:00,0.10770,0.10849,0.10757,0.10820,405,61,0,0,DOGECOIN
48,2024-10-04 11:30:00,0.10820,0.10854,0.10810,0.10839,442,61,0,top,DOGECOIN
49,2024-10-04 11:45:00,0.10842,0.10854,0.10780,0.10780,305,60,0,0,DOGECOIN


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 10:45:00,64.41,64.46,64.34,64.36,182,1,0,top,LITECOIN
46,2024-10-04 11:00:00,64.37,64.44,64.26,64.29,165,1,0,0,LITECOIN
47,2024-10-04 11:15:00,64.26,64.54,64.25,64.49,135,1,0,bottom,LITECOIN
48,2024-10-04 11:30:00,64.48,64.58,64.40,64.41,179,1,0,top,LITECOIN
49,2024-10-04 11:45:00,64.41,64.54,64.27,64.28,126,0,0,0,LITECOIN


...

,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 13:15:00,25.44,25.44,25.32,25.32,54,11,0,bottom,AVALANCH
46,2024-10-04 13:30:00,25.33,25.48,25.33,25.46,73,11,0,0,AVALANCH
47,2024-10-04 13:45:00,25.46,25.48,25.36,25.37,61,11,0,0,AVALANCH
48,2024-10-04 14:00:00,25.38,25.49,25.36,25.48,78,11,0,0,AVALANCH
49,2024-10-04 14:15:00,25.49,25.54,25.45,25.45,92,11,0,0,AVALANCH


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 13:30:00,0.52263,0.52492,0.52260,0.52465,410,21,0,0,RIPPLE
46,2024-10-04 13:45:00,0.52468,0.52521,0.52237,0.52335,376,25,0,top,RIPPLE
47,2024-10-04 14:00:00,0.52336,0.52364,0.52203,0.52237,450,23,0,0,RIPPLE
48,2024-10-04 14:15:00,0.52240,0.52411,0.52183,0.52301,541,20,0,0,RIPPLE
49,2024-10-04 14:30:00,0.52301,0.52305,0.52301,0.52305,3,40,0,0,RIPPLE


Waiting for 900.0 seconds until next 15-minute check.


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 13:30:00,551.25,552.53,551.03,552.18,1406,11,0,0,BNB
46,2024-10-04 13:45:00,552.18,552.61,551.70,552.24,1408,11,0,0,BNB
47,2024-10-04 14:00:00,552.24,552.99,552.04,552.86,1340,11,0,0,BNB
48,2024-10-04 14:15:00,552.86,553.29,552.30,552.35,1276,11,0,0,BNB
49,2024-10-04 14:30:00,552.35,552.45,552.35,552.41,5,11,0,0,BNB


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
45,2024-10-04 13:30:00,25.33,25.48,25.33,25.46,73,11,0,0,AVALANCH
46,2024-10-04 13:45:00,25.46,25.48,25.36,25.37,61,11,0,0,AVALANCH
47,2024-10-04 14:00:00,25.38,25.49,25.36,25.48,78,11,0,0,AVALANCH
48,2024-10-04 14:15:00,25.49,25.54,25.45,25.45,92,11,0,0,AVALANCH
49,2024-10-04 14:30:00,25.46,25.46,25.46,25.46,1,12,0,0,AVALANCH
